# Generate simplified documentation pages
Use data available in Houdini documentation.

In [1]:
import sys
import os
import os.path as op
import copy
import zipfile
import json
import yaml
import html
import htmltag
import bs4

HFS = 'C:/Program Files/Side Effects Software/Houdini 16.5.268'
sys.path.append(op.join(HFS, 'houdini/python2.7libs'))

from bookish.wikipages import parse_string as parse_wikipage

In [2]:
# Open vex.zip containing various helpcards including functions.
# Parse Wiki files into JSON using Bookish parser.
functions = {}

with zipfile.ZipFile(op.join(HFS, 'houdini/help/vex.zip')) as z:
    for path in z.namelist():
        if op.dirname(path) == 'functions':
            with z.open(path) as f:
                name = op.splitext(op.basename(path))[0]
                markup = f.read().decode()
                functions[name] = parse_wikipage(markup)

In [3]:
# Inspect Bookish types.
def types(node):
    '''Recursively extract all 'type' values from Bookish tree.'''
    if type(node) is dict:
        yield node['type']

        for k in node:
            if type(node[k]) is list:
                for i in node[k]:
                    for j in types(i):
                        yield j

def all_types(functions):
    for f in functions:
        for t in types(functions[f]):
            yield t


bookish_types = set(all_types(functions))
print(' '.join(sorted(bookish_types)))
print('Total:', len(bookish_types))

arg box bullet cell code dt em examples_section h link list note null ord para pre prop pxml related_section returns returnss root strong subtopics_section summary supertitle tip title ui usage var varg warning xml
Total: 34


In [4]:
# Include tools.
def make_common():
    '''
    Build a dictionary of bookish trees correspoinding to a specific
    _common#include like description of <geometry> argument.
    '''
    items = copy.deepcopy(functions['_common']['body'][1:])
    common = {}

    # Find all ids placed to second places in subunits and swap them
    # with first positions items holding human-readable strings.
    new_arg = False
    for i, item in enumerate(items):
        if new_arg:
            key = item['value']
            # Keep track of found keys.
            common[key] = []
            # Move key to starting position in the sequence of subunit.
            items[i-1], items[i] = key, items[i-1]

        # Update tracking flag.
        if item['indent'] == 0:
            new_arg = True
        else:
            new_arg = False

    # Distribute result using starting strings as keys
    # and next elements until next key as items.
    cur_key = None
    for item in items:
        if isinstance(item, str):
            cur_key = item
        else:
            common[cur_key].append(item)

    return common


include_common = make_common()


def get_include(key):
    '''Return include tree based on the input element.'''
    generic_includes = [
        '_space_args',
        '_adaptive_variadic',
        '_area_variadic',
        '_bias_variadic',
        '_derive_variadic',
        '_gather_variadic',
        '_imagefilter_variadic',
        '_lightmask_variadic',
        '_rayopts_variadic',
        'pbr_mask_h',
    ]
    if key.startswith('_common#'):
        return include_common[key.split('#')[1]  ]
    elif key in generic_includes:
        return functions[key]['body']
    return None

In [5]:
class Wrapper:
    '''Convert Bookish JSON structure to html.'''

    def __init__(self):
        # Trivial type handlers.
        from htmltag import body, code, div, em, h1, h2, p, span, strong

        self.root = lambda n: body(self(n['body']))

        self.arg = lambda n: div(code(self(n['text'])), _class='argument')
        self.bullet = lambda n: div(self(n['text']), _class='related')
        self.cell = lambda n: span(self(n['text']))
        self.code = lambda n: code(self(n['text']))
        self.em = lambda n: em(self(n['text']))
        self.para = lambda n: p(self(n['text']))
        self.strong = lambda n: strong(self(n['text']))
        self.summary = lambda n: p(self(n['text']), _class='summary')
        self.usage = lambda n: div(self(n['text']), _class='usage')
        self.var = lambda n: code(self(n['text']), _class='var')
        self.xml = lambda n: self(n['text'])

        self.h = lambda n: h2(self(n['text']))
        self.note = lambda n: h2('Note')
        self.tip = lambda n: h2('Tip')
        self.warning = lambda n: h2('Warning')
        self.returns = lambda n: h2('Returns')

        self.examples_section = lambda n: h2('Examples')
        self.related_section = lambda n: h2('Related')
        self.subtopics_section = lambda n: ''

        self.box = lambda n: ''
        self.keys = lambda n: ''
        self.list = lambda n: ''
        self.null = lambda n: ''
        self.ord = lambda n: ''
        self.pxml = lambda n: ''
        self.supertitle = lambda n: ''

        self.dt = self.arg
        self.item = self.bullet
        self.ui = self.strong
        self.varg = self.arg
        self.returnss = self.returns  # Patch someone's typo.
        
    def title(self, n):
        from htmltag import a, h1
        name = self(n['text'])
        url = f'https://www.sidefx.com/docs/houdini/vex/functions/{name}'
        return h1(a(name, href=url))

    def pre(self, n):
        from htmltag import HTML, code, div
        text = self(n['text']).strip()
        lines = text.split('\n')
        tags = [code(HTML(l), _class='codeline') for l in lines]
        return div(tags, _class='codeblock')

    def link(self, n):
        from htmltag import a

        if n['scheme'] == 'Include':
            key = n['value']

            # Fix some naming issues.
            key = key.replace('geofile_arg', 'geometry_output')
            key = key.replace('_lightmask_arg.txt', '_lightmask_variadic')
            key = key.replace('/vex/contexts/shading_contexts#lightmask', '_lightmask_variadic')
            key = key.replace('_lightmask_variadic#type=arg', '_lightmask_variadic')
            key = key.strip('/')
            
            include = get_include(key)
            
            if not include and '#' in key:
                return ''  # Do not handle includes from pages.

            if not include:
                print('Unknown include:', n)
                return ''

            return self(include)

        base = {
            'Exp': 'https://www.sidefx.com/docs/houdini/expressions/',
            'Hom': 'https://www.sidefx.com/docs/houdini/hom/hou/',
            'Hprop': 'https://www.sidefx.com/docs/houdini/props/mantra#',
            'Image': 'https://www.sidefx.com/docs/houdini',
            'Node': 'https://www.sidefx.com/docs/houdini/nodes/',
            'Vex': 'https://www.sidefx.com/docs/houdini/vex/functions/',
            'Cmd': 'https://www.sidefx.com/docs/houdini/commands/',
            'Wp': 'https://en.wikipedia.org/wiki/',
            None: 'https://www.sidefx.com/docs/houdini',
        }[n['scheme']]

        rest = n['value']
        if n['scheme'] == 'Hom':
            rest = rest.rsplit('.')[-1]

        content = n['value'] if not n['text'] else n['text']
        return a(self(content), href=base+rest)

    def prop(self, n):
        from htmltag import div, span
      
        # Currently used only to list deprecated functions.
        if n['name'] in ('index', 'status'):
            return ''

        # No need to specify that function generally works.
        elif n['name'] == 'context' and n['value'] == 'all':
            return ''
        
        # Tighten shading contexts.
        elif n['name'] == 'context' \
                and n['value'] == 'displace, fog, light, shadow, surface':
            t = span('shading contexts', _class='pillow')
            t = span(t, _class='padder')
            return t
        
        # Make boxes for tags and contexts.
        elif n['name'] in ('tags', 'context'):
            tags = []
            for t in n['value'].split(','):
                t = t.strip()
                t = span(t, _class='pillow')
                t = span(t, _class='padder')
                tags.append(t)
            return ''.join(tags)

        # Varargs stuff.
        elif (n['name'] == 'default') or \
                (n['name'] == 'type' and n['value'] not in ('vex', 'include')):
            t = span(n['value'], _class='pillow')
            t = span(t, _class='padder')
            return t

        # Common unuseful props.
        if n['name'] in ('type', 'group', 'id', 'redirect', 'minitoc'):
            return ''

        print('Unknown prop:', n)
        return ''

    def __call__(self, node):
        if isinstance(node, dict):
            return getattr(self, node['type'])(node)
        elif isinstance(node, list):
            return htmltag.HTML(''.join(self(i) for i in node))
        else:
            return html.escape(node, quote=False)

        
wrapper = Wrapper()


# Debug.
# print(yaml.dump(functions['_lightmask_variadic']))
# markup = wrapper(functions['trace'])
# soup = bs4.BeautifulSoup(markup, 'html.parser')
# print(soup.prettify())


# Process some functions.
# helpcards = {
#     'getderiv': wrapper(functions['getderiv']),
#     'degrees': wrapper(functions['degrees']),
#     'resample_linear': wrapper(functions['resample_linear']),
#     'accessframe': wrapper(functions['accessframe']),
#     'prim_normal': wrapper(functions['prim_normal']),
#     'intersect': wrapper(functions['intersect']),
#     'addattribute': wrapper(functions['addattribute']),
#     'attribsize': wrapper(functions['attribsize']),
#     'addattrib': wrapper(functions['addattrib']),
#     'foreach': wrapper(functions['foreach']),
#     'chattr': wrapper(functions['chattr']),
#     'pgfind': wrapper(functions['pgfind']),
#     'xyzdist': wrapper(functions['xyzdist']),
#     'irradiance': wrapper(functions['irradiance']),
#     'ptransform': wrapper(functions['ptransform']),
#     'addpointattrib': wrapper(functions['addpointattrib']),
#     'getlights': wrapper(functions['getlights']),
# }


# Process all functions.
drop = [
    'index',
    '_common',
    '_groups_en',
    '_space_args',
    '_adaptive_variadic',
    '_area_variadic',
    '_bias_variadic',
    '_derive_variadic',
    '_gather_variadic',
    '_imagefilter_variadic',
    '_lightmask_variadic',
    '_rayopts_variadic',
    'pbr_mask_h',
]
helpcards = {}
for f in functions:
    if f in drop:
        continue
    helpcards[f] = wrapper(functions[f])


# Dump on disk.
with open('helpcards.json', 'w') as f:
    json.dump(helpcards, f, indent=4)